<a href="https://colab.research.google.com/github/ocoropuj/PHYS434/blob/main/Homework_5_pvalue.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Homework 5


For a simple counting experiment, the expected background event is $b$ and the observed event is $n$.
The best estimator for signal event $s$ is:
$$s=n-b.$$

There are different metrics to evaluate discovery significance.

* Simplified Z0
$$ Z_{0, simple} = s/\sqrt{b}$$

* Asympototic Z0
$$ Z_{0, asymptotic} = \sqrt{2((s+b)\mathrm{ln} (1+s/b)-s)}$$

* Bayesian Z0
$$ p-value = \int_{n}^{\infty}\mathrm{Poisson}(k|b) dk$$.
$$Z_{0, Bayesian} =  \mathrm{Gauss_{1-sided}(p-value)} $$


In this exercise, we will implement each of the metric and compare consistency.


In [4]:
# @title Install Iminuit
!pip install iminuit


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 402.8/402.8 kB 1.5 MB/s eta 0:00:00


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import iminuit.minimize as minimize
from tqdm import tqdm
import scipy
from scipy.stats import poisson, norm



In [5]:
# Define test statistics q_0 for Frequentist approach
# 1. We require signal event s >0 for positive signal yield.
#    Therefore, the test statistics q_0 is 0 if N_obs <= Nb

def q0(N_obs, N_b):
    if N_obs <= N_b:
        return 0  # q_0 is 0 for N_obs <= N_b

    # Calculate the likelihood ratio for the null hypothesis
    null_hypothesis_likelihood = -2 * (N_obs * np.log(N_b) - N_b)

    # Calculate the likelihood ratio for the alternative hypothesis (signal plus background)
    # Here, we assume that the signal strength is equal to the observed events
    alternative_hypothesis_likelihood = -2 * (N_obs * np.log(N_obs) - N_obs)

    # Calculate q_0 as the difference between the null and alternative hypothesis likelihood ratios
    q0_out = null_hypothesis_likelihood - alternative_hypothesis_likelihood

    return q0_out


In [8]:
# 2. Compute two Poisson loglikelihood of
#    a) backgorund only model
#    b) signal+background model
#    Evaluate -2 log likelihood ratio between a) and b)

def log_likelihood_background_only(N_obs, N_b):
    # Poisson log-likelihood for the background-only model
    return N_obs * np.log(N_b) - N_b - np.sum(np.log(np.arange(1, N_obs + 1)))

def log_likelihood_signal_plus_background(N_obs, N_b, signal_strength):
    # Poisson log-likelihood for the signal+background model
    total_expected = N_b + signal_strength
    return N_obs * np.log(total_expected) - total_expected - np.sum(np.log(np.arange(1, N_obs + 1)))

def likelihood_ratio(N_obs, N_b, signal_strength):
    # Calculate -2 log likelihood ratio between signal+background and background-only
    log_likelihood_bg_only = log_likelihood_background_only(N_obs, N_b)
    log_likelihood_sig_plus_bg = log_likelihood_signal_plus_background(N_obs, N_b, signal_strength)

    return -2 * (log_likelihood_bg_only - log_likelihood_sig_plus_bg)


Implement four metrics:

In [ ]:
def SimplifiedZ0(N_obs, N_b):
    if N_obs <= N_b:
        return 0.0  # Z-score is 0 for N_obs <= N_b
    s = N_obs - N_b
    Zscore = s / np.sqrt(N_b)
    return Zscore

def AsymptoticZ0(N_obs, N_b):
    if N_obs <= N_b:
        return 0.0  # Z-score is 0 for N_obs <= N_b
    s = N_obs - N_b
    Zscore = np.sqrt(2 * ((s + N_b) * np.log(1 + s / N_b) - s))
    return Zscore


def BayesianZ0(N_obs,N_b):
    # Write your code here
    pvalue = 1-poisson.cdf(N_obs, N_b)
    Zscore= scipy.stats.norm.ppf(1-pvalue)
    return Zscore


Now, let's apply our code for numerical calculations.

Consider the case that backogrund only model with yields b=0.5 and observed events n=5.

Calclate discovery significance for each of the metric, respectively.

In [ ]:
Nobs=5
Nb=0.5

# Calculate discovery significance for Simplified Z0
SimplifiedZ0 = SimplifiedZ0(Nobs, Nb)

# Calculate discovery significance for Asymptotic Z0
AsymptoticZ0 = AsymptoticZ0(Nobs, Nb)

p_value = 1 - poisson.cdf(Nobs, Nb)
BayesianZ0 = norm.ppf(1 - p_value)

print("Simplified Z0:", SimplifiedZ0)
print("Asymptotic Z0:", AsymptoticZ0)
print("Bayesian Z0:", BayesianZ0)

Simplified Z0: 6.363961030678928
Asymptotic Z0: 3.7451102693966782
Bayesian Z0: 4.186492134133442


Describe the consistency between different metrics.

Write your answers here:

We find the following points:

1. All three metrics provide positive Z-scores indicating that there is a level of statistical significance in the data. And indicating a deviation "in the same direction" from the background.
2. The simplified $Z_{0,simple}$ provide the highest $Z$-score which indicates a larger value of $s$ since it grows linearly (Faster than the other methods)
